In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2 gradio

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr
import re

In [ ]:
# one can always add more models, of course

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
OPENAI_MODEL = "gpt-4o-mini"

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [ ]:
def dataset_generator(model, nature, shots, volume, language):

  examples = "Instruction: 'Make a random sentence.'\nAnswer: 'When I got home last night, I couldn't believe my eyes: All the pineapples had been removed from the pizza.'"
  system_message = "You are a random sentence generator. Generate 10 diverse English sentences."
  user_prompt = f"Generate 10 random English sentences, like so:\n{examples}"
  sentences = ""

  if language == "English":

    for shot in list(shots.keys()):
      examples += f"\nExample instruction: '{shot}'\nExample answer: '{shots[shot]}'\n"

    system_message = f"You are a state-of-the art linguistic dataset compiler. You are given a 'Type' of sentence to create. \
Within the bounds of that type, create {volume} diverse sentences with differing structures and lengths. Make the sentences plausible, \
but be creative in filling them with random concrete information, names, and data. Here are some examples for how to go about that:\n{examples}\n\
Just output one sentence per line. Do not comment or format yor output in any way, shape, or form."

    user_prompt = f"Generate {volume} English sentences of the following Type: {nature}. Just output one sentence per line. \
Do not comment or format yor output in any way, shape, or form."

  elif language == "German":

    for shot in list(shots.keys()):
      examples += f"\nAnweisung: '{shot}'\nAntwort: '{shots[shot]}'\n"

    system_message = f"Du bist ein weltklasse Datensatz-Sammler für Sprachdaten. Du erhältst einen 'Typ' von Sätzen, die du erstellen sollst. \
Im Rahmen dieses Typs, generiere {volume} untereinander verschiedene Sätze mit unterschiedlichen Satzlängen und -strukturen. Mache die Beispielsätze \
plausibel, aber fülle sie kreativ mit willkürlichen Informationen, Namen, und Daten aller Art. Hier sind ein paar Beispiel, wie du vorgehen sollst:\n{examples}\n\
Gib einfach einen Satz pro Zeile aus. Kommentiere oder formatiere deine Antwort in keinster Weise."

    user_prompt = f"Generiere {volume} deutsche Sätze des folgenden Typs: {nature}. Gib einfach einen Satz pro Zeile aus. \
Kommentiere oder formatiere deine Antwort in keiner Weise."

  elif language == "French":

    for shot in list(shots.keys()):
      examples += f"\nConsigne: '{shot}'\nRéponse: '{shots[shot]}'\n"

    system_message = f"Tu es un outil linguistique de pointe, à savoir, un genérateur de données linguistiques. Tu seras assigné un 'Type' de phrases à créer. \
Dans le cadre de ce type-là, crée {volume} phrases diverses, avec des structures et longueurs qui varient. Génère des phrases qui soient plausibles, \
mais sois créatif, et sers-toi de données, noms, et informations aléatoires pour rendre les phrases plus naturelles. Voici quelques examples comment faire:\n{examples}\n\
Sors une seule phrase par ligne. Ne formatte ni commente ta réponse en aucune manière que ce soit."

    user_prompt = f"S'il te plaît, crée {volume} phrases en français du Type suivant: {nature}. Sors une seule phrase par ligne. \
Ne formatte ni commente ta réponse en aucune manière que ce soit."

  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_prompt}
    ]

  if model == "Llama":

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

    tokenizer = AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)
    model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
    outputs = model.generate(inputs, max_new_tokens=10000)

    response  = tokenizer.decode(outputs[0])
    sentences = list(re.finditer("(?:<\|end_header_id\|>)([^<]+)(?:<\|eot_id\|>)", str(response), re.DOTALL))[-1].group(1)

  elif model == "OpenAI":
    response = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages)
    sentences = response.choices[0].message.content

  return sentences

In [ ]:
global data
data = ""

with gr.Blocks(
        css="""
    .red-button {
        background-color: darkred !important;
        border-color: red !important;
    }
    .blue-button {
        background-color: darkblue !important;
        border-color: blue !important;
    }
    .green-button {
        background-color: green !important;
        border-color: green !important;
    }
    """
) as view:
  with gr.Row():
    title = gr.HTML("<h1><big>D</big>ataset Generator <small>PLUS</small></h1><h2>for English, German, and French</h2>")
    subtitle = gr.HTML("<h3>Instructions:</h3><ol><li>Pick the language</li>\
<li>Select a model</li><li>Indicate how many sentences you need</li>\
<li>Describe the type of sentence you're looking for</li><li>Give up to three examples of the desired output sentence, and describe each of them briefly</li>\
<li>Hit <q>Create Dataset</q></li>\
<li>Save the output (.txt) to your Google Drive</li>")
  with gr.Row():
    language_choice = gr.Dropdown(choices=["English", "German", "French"], label="Select language", value="English", interactive=True)
    model_choice    = gr.Dropdown(choices=["Llama", "OpenAI"], label="Select model", value="Llama", interactive=True)
    volume = gr.Textbox(label="Required number of sentences", interactive=True)
  with gr.Row():
    typeInput = gr.Textbox(label="Short description of the kind of sentence you need", interactive=True)
  with gr.Row():
    sentence_1    = gr.Textbox(label="Example sentence 1", interactive=True)
    instruction_1 = gr.Textbox(label="Description", interactive=True)
  with gr.Row():
    sentence_2    = gr.Textbox(label="Example sentence 2", interactive=True)
    instruction_2 = gr.Textbox(label="Description", interactive=True)
  with gr.Row():
    sentence_3    = gr.Textbox(label="Example sentence 3", interactive=True)
    instruction_3 = gr.Textbox(label="Description", interactive=True)
  with gr.Row():
    liveSentences = gr.Markdown(
        value='<div style="color: #999; padding: 10px;">Your sentences will be displayed here …</div>',
        label="Generated sentences:",
         min_height=60,
         max_height=200
        )
  with gr.Row():
    generate = gr.Button(value="Generate sentences", elem_classes="blue-button")
  with gr.Row():
    clear = gr.Button(value="Clear everything", elem_classes="red-button")
  with gr.Row():
    outputPath  = gr.Textbox(label="Specify the desired name and location on your Google Drive for the sentences (plain text) to be saved", interactive=True)
  with gr.Row():
    save  = gr.Button(value="Save generated data", elem_classes="blue-button")

  def generateSentences(typeInput, s1, i1, s2, i2, s3, i3, volume, language, model):
    global data
    nature = ""
    shots = {}
    amount = int(volume) if re.search("^[0-9]+$", volume) is not None else 10

    if typeInput != None:
      nature = typeInput
    else:
      nature = "Random sentences of mixed nature"

    if s1 != None:
      if i1 != None:
        shots[i1] = s1
      else:
        shots["A medium-long random sentence about anything"] = s1
    else:
      shots["A medium-long random sentence about anything"] = "Paul, waking up out of his half-drunken haze, clearly couldn't tell left from right and ran right into the door."

    if s2 != None:
      if i2 != None:
        shots[i2] = s2
      else:
        shots["A medium-long random sentence about anything"] = s2

    if s3 != None:
      if i3 != None:
        shots[i3] = s3
      else:
        shots["A medium-long random sentence about anything"] = s3

    sentences = dataset_generator(model, nature, shots, amount, language)
    data = sentences

    return sentences

  def saveData(path):
    global data
    drive.mount("/content/drive")

    dir_path = os.path.dirname("/content/drive/MyDrive/" + path)

    if not os.path.exists(dir_path):
      os.makedirs(dir_path)

    with open("/content/drive/MyDrive/" + path, "w", encoding="utf-8") as f:
      f.write(data)

  generate.click(generateSentences, inputs=[typeInput, sentence_1, instruction_1, sentence_2, instruction_2, sentence_3, instruction_3, volume, language_choice, model_choice], outputs=liveSentences)
  clear.click(
      lambda: [
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value=""),
          gr.update(value='<div style="color: #999; padding: 10px;">Your sentences will be displayed here …</div>'),
          gr.update(value=""),
          gr.update(value="Save generated data", elem_classes="blue-button")],
      None,
      [volume, typeInput, sentence_1, instruction_1, sentence_2, instruction_2,
         sentence_3, instruction_3, liveSentences, outputPath, save],
      queue=False
      )
  save.click(saveData, inputs=outputPath, outputs=None).then(lambda: gr.update(value="Your data has been saved", elem_classes="green-button"), [], [save])

view.launch(share=True) #, debug=True)